# Introduction

In this tutorial, we'll demonstrate how to leverage a sample dataset stored in Azure Cosmos DB to ground OpenAI models. We'll do this taking advantage of Azure Cognitive Search's vector similarity search functionality. In the end, we'll create an interatice chat session with the GPT-3.5 completions model to answer questions about Azure services informed by our dataset. This process is known as Retrieval Augmented Generation, or RAG.

This tutorial borrows some code snippets and example data from the Azure Cognitive Search Vector Search demo repository: https://github.com/Azure/cognitive-search-vector-pr/. 

# Preliminaries <a class="anchor" id="preliminaries"></a>
First, let's start by installing the packages that we'll need later. Note that for `azure-search-documents`, you'll need to install a specific version from the preview azure sdk. 

In [ ]:
! pip install numpy
! pip install openai
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos
! pip install tenacity
! pip install --index-url=https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/ azure-search-documents==11.4.0a20230509004

In [ ]:
import json
import datetime
import time

from azure.core.exceptions import AzureError
from azure.core.credentials import AzureKeyCredential
from azure.cosmos import exceptions, CosmosClient, PartitionKey
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.models import Vector
from azure.search.documents.indexes.models import (
    IndexingSchedule,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchField,
    SearchFieldDataType,
    SearchableField,
    SemanticConfiguration,
    SimpleField,
    PrioritizedFields,
    SemanticField,
    SemanticSettings,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    SearchIndexerDataSourceConnection
)

import openai
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [ ]:
load_dotenv()  

# Configure connections
config_file = open(file="../config/config.json", mode="r")
config = json.load(config_file)
config_file.close()

cosmosdb_endpoint = config['cosmos_db_api_endpoint']
cosmosdb_key = config['cosmos_db_api_key']
cosmosdb_connection_str = config['cosmos_db_connection_string']

cog_search_endpoint = config['cognitive_search_api_endpoint']
cog_search_key = config['cognitive_search_api_key']

openai.api_type = config['openai_api_type']
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_endpoint']
openai.api_version = config['openai_api_version']
embeddings_deployment = config['openai_embeddings_deployment']
completions_deployment = config['openai_completions_deployment']

## Create an Azure Cosmos DB NoSQL resource<a class="anchor" id="cosmosdb"></a>
Let's start by creating an Azure Cosmos DB NoSQL Resource following this quick start guide: https://learn.microsoft.com/azure/cosmos-db/nosql/quickstart-portal

Then copy the connection details (endpoint, key, and connection string) into the config.json file.

## Azure Cognitive Search <a class="anchor" id="cognitivesearch"></a>

Next, let's create an Azure Cognitive Search resource following this quick start: https://learn.microsoft.com/azure/search/search-create-service-portal

Now copy the connection details (endpoint, key, and connection string) into the config.json file.

## Azure OpenAI <a class="anchor" id="azureopenai"></a>

Finally, let's setup our Azure OpenAI resource Currently, access to this service is granted only by application. You can apply for access to Azure OpenAI by completing the form at https://aka.ms/oai/access. Once you have access, complete the following steps:

- Create an Azure OpenAI resource following this quickstart: https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal
- Deploy a `completions` and `embeddings` model 
    - For more information on `completions`, go here: https://learn.microsoft.com/azure/ai-services/openai/how-to/completions
    - For more information on `embeddings`, go here: https://learn.microsoft.com/azure/ai-services/openai/how-to/embeddings
- Copy the endpoint, key, deployment names for (embeddings model, completions model) into the config.json file.

# Load data and create embeddings <a class="anchor" id="loaddata"></a>
Here we'll load a sample dataset containing descriptions of Azure services. Then we'll user Azure OpenAI to create vector embeddings from this data.

In [ ]:
# Load text-sample.json data file
data_file = open(file="../data/text-sample.json", mode="r")
# data_file = open(file="data/sample_text_w_embeddings.json", mode="r") # load this file instead if embeddings were previously created and saved.
data = json.load(data_file)
data_file.close()

In [ ]:
# Take a peek at one data item
print(data[0])

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
def generate_embeddings(text):
    '''
    Generate embeddings from string of text.
    This will be used to vectorize data and user input for interactions with Azure OpenAI.
    '''
    response = openai.Embedding.create(
        input=text, engine="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    time.sleep(0.5) # rest period to avoid rate limiting on AOAI for free tier
    return embeddings

In [ ]:
# Generate embeddings for title and content fields
for item in data:
    title = item['title']
    content = item['content']
    title_embeddings = generate_embeddings(title)
    content_embeddings = generate_embeddings(content)
    item['titleVector'] = title_embeddings
    item['contentVector'] = content_embeddings
    item['@search.action'] = 'upload'

# Save embeddings to sample_text_w_embeddings.json file
with open("../data/text-sample_w_embeddings.json", "w") as f:
    json.dump(data, f)

In [ ]:
# # If embeddings have already been created, load this file.
# data_file = open(file="data/text-sample_w_embeddings.json", mode="r")
# data = json.load(data_file)
# data_file.close()

# Upload data to Azure Cosmos DB

In [ ]:
# Create the client to interact with the Azure Cosmos DB resource
client = CosmosClient(cosmosdb_endpoint, cosmosdb_key)

In [ ]:
# Create a database in Azure Cosmos DB.
try:
    database = client.create_database_if_not_exists(id="VectorSearchTutorial")
    print(f"Database created: {database.id}")

except exceptions.CosmosResourceExistsError:
    print("Database already exists.")

In [ ]:
# Create a container in Azure Cosmos DB.
try:
    partition_key_path = PartitionKey(path="/id")
    container = database.create_container_if_not_exists(
        id="AzureServices",
        partition_key=partition_key_path,
        offer_throughput=400,
    )
    print(f"Container created: {container.id}")

except exceptions.CosmosResourceExistsError:
    print("Container already exists.")

In [ ]:
# Create data items for every entry in the dataset, insert them into the database and collection specified above.
for data_item in data:
    try:
        container.create_item(body=data_item)
    
    except exceptions.CosmosResourceExistsError:
        print("Data item already exists.")

## Create a search index in Cognitive Search 

Let's create the Search Index over all fields we have in our Azure Cosmos DB collection. Learn more about Azure Cognitive Search Indexes [here](https://learn.microsoft.com/azure/search/search-how-to-create-search-index).

In [ ]:
# Create index

cog_search_cred = AzureKeyCredential(cog_search_key)
index_name = "cosmosdb-vector-search-index"

# Create a search index and define the schema (names, types, and parameters)
index_client = SearchIndexClient(
    endpoint=cog_search_endpoint, credential=cog_search_cred)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="title", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True),
    SearchableField(name="content", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True),
    SearchableField(name="category", type=SearchFieldDataType.String,
                    filterable=True, searchable=True, retrievable=True),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, dimensions=1536, vector_search_configuration="my-vector-config"),
]

# Configure vector search
vector_search = VectorSearch(
    algorithm_configurations=[
        VectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            hnsw_parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 1000,
                "metric": "cosine"
            }
        )
    ]
)

# Configure semantic search. This will allow us to conduct semantic or hybrid search (with vector search) later on if desired.
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="title"),
        prioritized_keywords_fields=[SemanticField(field_name="category")],
        prioritized_content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

## Create an indexer to pull data from Cosmos DB into Cognitive Search

Now we'll create the indexer, which will retrieve data from our Azure Cosmos DB resource. Learn more about Azure Cognitive Search Indexers [here](https://learn.microsoft.com/azure/search/search-howto-create-indexers)

In [ ]:
# Create indexer

def _create_datasource():
    # Here we create a datasource. 
    ds_client = SearchIndexerClient(cog_search_endpoint, cog_search_cred)
    container = SearchIndexerDataContainer(name="AzureServices")
    data_source_connection = SearchIndexerDataSourceConnection(
        name="cosmosdb-tutorial-indexer", type="cosmosdb", connection_string=cosmosdb_connection_str, container=container
    )
    data_source = ds_client.create_or_update_data_source_connection(data_source_connection)
    return data_source

ds_name = _create_datasource().name

indexer = SearchIndexer(
        name="cosmosdb-tutorial-indexer",
        data_source_name=ds_name,
        target_index_name=index_name)

indexer_client = SearchIndexerClient(cog_search_endpoint, cog_search_cred)
indexer_client.create_or_update_indexer(indexer)  # create the indexer

result = indexer_client.get_indexer("cosmosdb-tutorial-indexer")
print(result)

# Run the indexer we just created.
indexer_client.run_indexer(result.name)


Now that we have setup our resources, data, and configured Azure Cognitive Search to index data from Azure Cosmos DB, let's try performing a vector similarity search.

In [ ]:
# Simple function to assist with vector search
def vector_search(query):
    search_client = SearchClient(cog_search_endpoint, index_name, cog_search_cred)  
    results = search_client.search(  
        search_text="",  
        vector=Vector(value=generate_embeddings(query), k=3, fields="contentVector"),  
        select=["title", "content", "category"] 
    )
    return results

Let's run a test query below.

In [ ]:
query = "tools for software development"  
results = vector_search(query)
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

# Q&A over the data with GPT-3.5

Finally, we'll create a helper function to feed prompts into the `Completions` model. Then we'll create interactive loop where you can pose questions to the model and receive information grounded in your data.

In [ ]:
#This function helps 

def generate_completion(prompt):
    system_prompt = '''
    You are an intelligent assistant for Microsoft Azure services.
    You are designed to provide helpful answers to user questions about Azure services given the information about to be provided.
        - Only answer questions related to the information provided below, provide 3 clear suggestions.
        - If you're unsure of an answer, you can say ""I don't know"" or ""I'm not sure"" and recommend users search themselves."
    '''

    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input},
    ]

    for item in results:
        messages.append({"role": "system", "content": prompt['content']})

    response = openai.ChatCompletion.create(engine=completions_deployment, messages=messages)
    
    return response

In [ ]:
# Create a loop of user input and model output. You can now perform Q&A over the sample data!

user_input = ""
print("*** Please ask your model questions about Azure services. Type 'end' to end the session.\n")
user_input = input("Prompt: ")
while user_input.lower() != "end":
    results_for_prompt = vector_search(user_input)
    completions_results = generate_completion(results_for_prompt)
    print("\n")
    print(completions_results['choices'][0]['message']['content'])
    user_input = input("Prompt: ")
